In [ ]:
import torch
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from datasets import load_dataset
from trl import SFTTrainer
from accelerate import Accelerator
import os


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True" # 메모리 조각 방지
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_MODEL = "./DeepSeek-R1-Distill-Llama-8B"

In [ ]:
dataset = load_dataset("code_search_net", "python")  # 언어 선택 가능 (ex: python, java)
train_data = dataset["train"]
valid_data = dataset["validation"]

In [ ]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    #target_modules=["q_proj", "o_proj", "k_proj", "v_proj"],  # 가중치 적용할 레이어 빠른학습
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # 가중치 적용할 레이어 정확도
    #target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "embed_tokens", "lm_head"],  # 가중치 적용할 레이어 도메인 전체 학습
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# 4bit 양자화 설정 - QLoRA로 해야 함
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", #nf4
    bnb_4bit_use_double_quant=True, #True
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token  # 패딩 토큰 설정

# 4-bit 양자화된 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    #device_map="sequential",
    device_map="auto",
    quantization_config=bnb_config  # 4-bit 설정 적용
)


In [ ]:
# RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn 에러
model.enable_input_require_grads() # get_input_embeddings().weight.requires_grad = True 
# LoRA 적용
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# LoRA가 적용된 레이어만 학습 (모델 파라미터 freeze)
for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True

In [ ]:
def tokenize_function(examples):
    # 함수 코드와 Docstring을 하나의 입력으로 결합
    combined_texts = [
        f"{doc}\n\n{code}" for doc, code in zip(examples["func_documentation_string"], examples["func_code_string"])
    ]
    
    tokenized = tokenizer(
        combined_texts,  
        truncation=True, 
        padding="max_length", 
        max_length=512
    )
    
    tokenized["labels"] = torch.tensor(tokenized["input_ids"])  # ✅ `torch.tensor()` 사용
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="./deepseek-code-doc-lora-V2",
    dataloader_pin_memory=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_checkpointing=False,  # T: GPU 메모리 절약
    gradient_accumulation_steps=2,  # 작은 배치 크기 보완
    num_train_epochs=1,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir="./code_logs",
    fp16=True,  # 16-bit 연산
    optim="paged_adamw_8bit",
    ddp_find_unused_parameters=False  # DDP 활성화
)

In [ ]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [ ]:
model.train()
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

trainer.train()

In [ ]:
# LoRA 가중치만 저장
model.save_pretrained("./deepseek-code-doc-lora-V2", safe_serialization=True)
tokenizer.save_pretrained("./deepseek-code-doc-lora-V2")

trainer.state.save_to_json("./deepseek-code-doc-lora-V2/trainer_state.json")

In [ ]:
# 4bit 양자화 설정 - QLoRA로 해야 함
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", #nf4
    bnb_4bit_use_double_quant=True, #True
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token  # 패딩 토큰 설정

# 4-bit 양자화된 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    #device_map="sequential",
    device_map="auto",
    quantization_config=bnb_config  # 4-bit 설정 적용
)


# LoRA 적용된 모델 불러오기
lora_model = PeftModel.from_pretrained(model, "./deepseek-code-doc-lora-V2")

# LoRA 병합 (LoRA 가중치를 원본 모델에 합침)
merged_model = lora_model.merge_and_unload()

# 병합된 모델 저장 (이제 일반 모델처럼 사용 가능)
merged_model.save_pretrained("./deepseek-code-doc-merged-V2")
tokenizer.save_pretrained("./deepseek-code-doc-merged-V2")


In [ ]:
accelerator = Accelerator(mixed_precision="fp16")

In [ ]:
# 4bit 양자화 설정 - QLoRA로 해야 함
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", #nf4
    bnb_4bit_use_double_quant=True, #True
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True
)

# 기본 모델 로드 (LoRA 적용 전 원본 모델)
lora_model = AutoModelForCausalLM.from_pretrained("./deepseek-code-doc-merged-V2", device_map='auto', quantization_config=bnb_config)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("./deepseek-code-doc-merged-V2")

In [ ]:
# Accelerator 준비 (mixed precision 적용)
lora_model = accelerator.prepare(lora_model)

In [ ]:
# 텍스트 생성 파이프라인 (dispatch_model 없이 바로 사용)
pipe = pipeline(
    "text-generation",
    model=lora_model,
    tokenizer=tokenizer,
    max_new_tokens=256,  # ✅ 토큰 수 절약
    device_map='auto',   # ✅ 여러 GPU 자동 분배
    #offload_folder="./offload",  # CPU로 오프로드하여 GPU 메모리 절약
    #offload_state_dict=True  # 상태 딕셔너리 오프로드
)

# 프롬프트 설정
document = f"""
1 + 1 코드를 만들어 주세요
"""

messages = [
    {"role": "user", "content": f"""
    1. 코드보여줘
    2. 반드시 한국어로 설명해
    Document:
    {document}
    """},
]

prompt = pipe.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

# 생성
with torch.no_grad():
    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.5,  # 샘플링의 다양성을 높이기 위한 온도 설정
        top_k=40,  # 가장 높은 확률을 가진 top k 개의 단어만 고려
        top_p=0.8,  # 누적 확률이 80%인 단어들만 고려
        repetition_penalty=1.2,
        add_special_tokens=True,
        eos_token_id=[  
            pipe.tokenizer.eos_token_id,
            pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
        ]
    )

# 결과 출력
print(outputs[0]["generated_text"])

In [ ]:
print("EOS Token ID:", tokenizer.eos_token_id)
print("모든 특수 토큰:", tokenizer.all_special_tokens)
